<h1> MILK        SERVIDOR </h1>

<p>Importar Librerias</p>

In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
import csv
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from multiprocessing import Process
from multiprocessing import Pool

<p>Cargamos los datos</p>

In [2]:
df = pd.read_csv('/home/paul/Documentos/Nuevas Tablas Profeco 2016/xxx/all_milk_xxx.csv', encoding='utf8', nrows=10,
                 names=['producto','presentacion','marca','categoria','precio','fechaRegistro','cadenaComercial', 
                          'giro','nombreComercial','direccion','estado','municipio'])

<p>Reordenamos el dataFrame</p>

In [3]:
df = df[['producto','presentacion','marca','categoria','cadenaComercial','giro','nombreComercial','direccion','estado',
        'municipio','fechaRegistro','precio']]

In [4]:
le = LabelEncoder()

In [5]:
df['producto'] = le.fit_transform(df['producto'].astype(str))

In [6]:
df['presentacion'] = le.fit_transform(df['presentacion'].astype(str))
df['marca'] = le.fit_transform(df['marca'].astype(str))
df['categoria'] = le.fit_transform(df['categoria'].astype(str))
df['cadenaComercial'] = le.fit_transform(df['cadenaComercial'].astype(str))
df['giro'] = le.fit_transform(df['giro'].astype(str))
df['nombreComercial'] = le.fit_transform(df['nombreComercial'].astype(str))
df['direccion'] = le.fit_transform(df['direccion'].astype(str))
df['estado'] = le.fit_transform(df['estado'].astype(str))
df['municipio'] = le.fit_transform(df['municipio'].astype(str))

In [7]:
X_train = df.iloc[:, 0:10].values
y_train = df.iloc[:, 10].values

<b>Comenzamos a Procesar los Datos</b>
<p>Convertimos las string a tipo datetime con pandas</p>

In [10]:
df['fechaRegistro'] = pd.to_datetime(df['fechaRegistro'], format='%Y-%m-%d %H:%M')

<b>Comenzando a tratar las fechas</b>
<p>Creamos nuevas variables a partir de las fechas mediante pandas</p>

In [11]:
datetimes = df['fechaRegistro']
over_time = pd.DataFrame({'year': datetimes.dt.year,
                          "month": datetimes.dt.month,
                          "day": datetimes.dt.day,
                          "dayofweek": datetimes.dt.dayofweek,
                          "dayofyear": datetimes.dt.dayofyear,
                          "weekofyear": datetimes.dt.weekofyear,
                          "quarter": datetimes.dt.quarter,
                         })

In [12]:
len(datetimes)

10

In [13]:
time = []
for i in range(0, len(datetimes)):
    time.append([over_time['year'][i], over_time['month'][i], over_time['day'][i], over_time['dayofweek'][i], 
                 over_time['dayofyear'][i], over_time['weekofyear'][i], over_time['quarter'][i]])

In [21]:
time2=[]
def time_(start, end):
    for i in range(start, end):
        time2.append([over_time['year'][i], over_time['month'][i], over_time['day'][i], over_time['dayofweek'][i], 
                     over_time['dayofyear'][i], over_time['weekofyear'][i], over_time['quarter'][i]])
    

In [22]:
time_(0,5)

In [23]:
time2


[[2016, 4, 18, 0, 109, 16, 2],
 [2016, 4, 18, 0, 109, 16, 2],
 [2016, 4, 18, 0, 109, 16, 2],
 [2016, 4, 18, 0, 109, 16, 2],
 [2016, 4, 18, 0, 109, 16, 2]]

In [27]:
print(over_time['year'],over_time['month'], over_time['day'], over_time['dayofweek'])

0    2016
1    2016
2    2016
3    2016
4    2016
5    2016
6    2016
7    2016
8    2016
9    2016
Name: year, dtype: int64 0    4
1    4
2    4
3    4
4    4
5    4
6    4
7    4
8    4
9    4
Name: month, dtype: int64 0    18
1    18
2    18
3    18
4    18
5    18
6    18
7    18
8    18
9    18
Name: day, dtype: int64 0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: dayofweek, dtype: int64


In [28]:
with Pool(processes=2) as pool:
    processed_time1 = pool.map(time_,0,5)
    processed_time2 = pool.map(time_,5,10)

TypeError: 'int' object is not iterable

<b>Metemos todas las variables a columnas indicadas por las cabeceras de la base de datos</b>
<p>Hacemos uso de diccionarios y de nuevo de el DataFrame de pandas</p>

In [ ]:
column = {}
for i in range(0, len(df.axes[1])-2):
    column[df.axes[1][i]] = X_train[:, i]

<b>Binarizamos todas las columnas que son strings y se trataran como etiquetas</b>
<ul>
  <li>Hacemos uso de Herramientas de sklearn.preprocessing</li>
  <li>Seguimos usando el DataFrame de pandas para el manejo de las columnas</li>
</ul>

In [ ]:
binarized_column = {}
lb = LabelBinarizer()
for i in range(0, len(df.axes[1])-2):
    binarized_column[df.axes[1][i]] = lb.fit_transform(column[df.axes[1][i]]).tolist()

<b>Creamos la matriz de Diseño</b>

In [ ]:
Xdesing = []
for i in range(0, len(X_train)):
    Xdesing.append(binarized_column['producto'][i] + binarized_column['presentacion'][i]+ binarized_column['marca'][i] 
                  + binarized_column['categoria'][i] + binarized_column['cadenaComercial'][i] + binarized_column['giro'][i]
                  + binarized_column['direccion'][i] + binarized_column['estado'][i] + binarized_column['municipio'][i] 
                  + time[i])
Xdesing = np.asarray(Xdesing)
y = np.asarray(y_train)

In [ ]:
Xdesing.shape

In [ ]:
import csv
with open('/home/paul/Documentos/Nuevas Tablas Profeco 2016/Xdesings/Xdesing_milk.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in Xdesing:
        spamwriter.writerow(row)